- https://nijianmo.github.io/amazon/index.html
- https://jmcauley.ucsd.edu/data/amazon/ 
- https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV

In [1]:
prfx = 'prep_20210302B1'

# get data

    download from http://localhost:8080/notebooks/git/product-category/notebooks/secrets_get_amazon_data.ipynb

In [2]:
import pandas as pd
import gzip
import json

In [3]:
pdata = "/data/git/product-category/data/amazon_meta_data"
!ls -hl {pdata}
fnms = !ls {pdata}
fnms = [o for o in fnms if o.endswith('.json.gz')]
domains = [o.strip('meta_').strip('.json.gz') for o in fnms]
dmn2fnm = dict(zip(domains, fnms))
dmn2fnm

total 13G
-rw-rw-r-- 1 ubuntu ubuntu   30M Dec  2 00:07 meta_AMAZON_FASHION.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  9.6M Aug  7  2020 meta_All_Beauty.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   57M Aug  7  2020 meta_Appliances.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  125M Aug  7  2020 meta_Arts_Crafts_and_Sewing.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Aug  7  2020 meta_Automotive.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Books.json.gz.1
-rw-rw-r-- 1 ubuntu ubuntu  153M Aug  7  2020 meta_CDs_and_Vinyl.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  333M Aug  7  2020 meta_Cell_Phones_and_Accessories.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.5G Dec  2 00:18 meta_Clothing_Shoes_and_Jewelry.json.gz
-rw-rw-r-- 1 ubuntu ubuntu   12M Aug  7  2020 meta_Digital_Music.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  1.2G Aug  7  2020 meta_Electronics.json.gz
-rw-rw-r-- 1 ubuntu ubuntu  37

{'AMAZON_FASHION': 'meta_AMAZON_FASHION.json.gz',
 'All_Beauty': 'meta_All_Beauty.json.gz',
 'Appliance': 'meta_Appliances.json.gz',
 'Arts_Crafts_and_Sewi': 'meta_Arts_Crafts_and_Sewing.json.gz',
 'Automotive': 'meta_Automotive.json.gz',
 'Book': 'meta_Books.json.gz',
 'CDs_and_Vinyl': 'meta_CDs_and_Vinyl.json.gz',
 'Cell_Phones_and_Accessorie': 'meta_Cell_Phones_and_Accessories.json.gz',
 'Clothing_Shoes_and_Jewelry': 'meta_Clothing_Shoes_and_Jewelry.json.gz',
 'Digital_Music': 'meta_Digital_Music.json.gz',
 'Electronic': 'meta_Electronics.json.gz',
 'Gift_Card': 'meta_Gift_Cards.json.gz',
 'Grocery_and_Gourmet_Food': 'meta_Grocery_and_Gourmet_Food.json.gz',
 'Home_and_Kitche': 'meta_Home_and_Kitchen.json.gz',
 'Industrial_and_Scientific': 'meta_Industrial_and_Scientific.json.gz',
 'Kindle_Store': 'meta_Kindle_Store.json.gz',
 'Luxury_Beauty': 'meta_Luxury_Beauty.json.gz',
 'Magazine_Subscripti': 'meta_Magazine_Subscriptions.json.gz',
 'Movies_and_TV': 'meta_Movies_and_TV.json.gz',
 

In [4]:
KEYS2USE = set(['category', 'description', 'title', 'brand', 'feature', 'asin'])

In [5]:
# https://colab.research.google.com/drive/1Zv6MARGQcrBbLHyjPVVMZVnRWsRnVMpV#scrollTo=7igYuRaV4bF7
    
### load the meta data
def get_meta_data(domain, nrows=None):
    keys2use = KEYS2USE
    fnm = dmn2fnm[domain]
    data = []
    with gzip.open(f'{pdata}/{fnm}') as f:
        for i,l in enumerate(f):
            dat = json.loads(l.strip())
            dat = {k:v for k,v in dat.items() if k in keys2use}
            data.append(dat)
            if nrows and i>nrows: break

    # total length of list, this number equals total number of products
#     print("len(data)", len(data))

    # convert list into pandas dataframe
    df = pd.DataFrame.from_dict(data)
#     print("len(df)", len(df))

    ### remove rows with unformatted title (i.e. some 'title' may still contain html style content)
    df3 = df.fillna('')
    df4 = df3[df3.title.str.contains('getTime')] # unformatted rows
    df5 = df3[~df3.title.str.contains('getTime')] # filter those unformatted rows
#     print("len(df4)", len(df4))
#     print("len(df5)", len(df5))

    return df5

In [6]:
df = get_meta_data(domains[3], nrows=10000)
df.head(3)

,category,description,title,brand,feature,asin
0,"[Arts, Crafts & Sewing, Sewing, Trim & Embelli...",[The patch features the sweaty masculine hands...,You Son of a Bitch! 1987 Embroidered Patch,Honchosfx,[You son of a bitch patch - exclusive to Honch...,6665560953
1,"[Arts, Crafts & Sewing, Crafting, Paper & Pape...","[With 5 packs stars folding paper, each pack a...",Origami Stars Papers Package 1H (5 packs),,[],7000000376
2,"[Arts, Crafts & Sewing, Painting, Drawing & Ar...",[],Yi De Ge Chinese Calligraphy Sumi Drawing Blac...,MasterChinese,[],7000001089


# preprocess data

## concate

In [7]:
dfs = []
for dmn in domains:
    df = get_meta_data(dmn, nrows=None)
    if set(df.columns)!= KEYS2USE: continue
    df = df[df.category.apply(len)>0]
    if len(df)==0: continue
    print(dmn, df.shape)
    df['domain'] = dmn
    display(df.sample(2))
    dfs.append(df)

Appliance (29639, 6)


,category,description,title,brand,feature,asin,domain
28278,"[Appliances, Parts & Accessories, Refrigerator...","[This is an O.E.M. Authorized part, fits with ...",Samsung DA97-07150A Assembly Water Tank,Samsung,"[This is an O.E.M. Authorized part, Fits with ...",B01AQI0LJA,Appliance
8251,"[Appliances, Parts & Accessories, Range Parts ...",[The Prime Heavy Duty RV Power Cord enables an...,"Prime RV30925 RV Power Cord, 25 ft. with handl...",Prime,"[25 ft., Enables an RV with a 30 Amp power con...",B005IZ10T2,Appliance


Arts_Crafts_and_Sewi (284889, 6)


,category,description,title,brand,feature,asin,domain
154062,"[Arts, Crafts & Sewing, Scrapbooking & Stampin...",[Little B Dimensional Stickers make it easy to...,"Little B 100137 Dimensional Stickers, Mini, Bu...",Little B,[Dimensional embellishments bring unique and i...,B00HI4CC98,Arts_Crafts_and_Sewi
109457,"[Arts, Crafts & Sewing, Scrapbooking & Stampin...",[Make new cards for all occasions with the Cre...,Cricut 2001984 Project Creative Cards Cartridge,Cricut,[The most cost-effective way to acquire images...,B00B9TWNUG,Arts_Crafts_and_Sewi


Automotive (897587, 6)


...

,category,description,title,brand,feature,asin,domain
250656,"[Tools & Home Improvement, Building Supplies, ...",[Speedi-Products Round Pipe Saddle is a energy...,Speedi-Products SM-SDL90 10 10-Inch Diameter R...,Speedi-Products,"[90 Degree configuration, Air Tight Foam Rubbe...",B0085V036C,Tools_and_Home_Improvement
512307,"[Tools & Home Improvement, Lighting & Ceiling ...",[],Brightech - Maxwell LED Shelf Floor Lamp &ndas...,,[],B016UWXKXA,Tools_and_Home_Improvement


Toys_and_Game (571640, 6)


,category,description,title,brand,feature,asin,domain
38338,"[Toys & Games, Action Figures & Statues, Actio...",[Who will win the ultimate battle - Darth Vade...,AttackTix Battle Figure Game - Star Wars vs. T...,Hasbro,[Lead a squad into battle and knock down your ...,B000ELIXFY,Toys_and_Game
209488,"[Toys & Games, Dolls & Accessories, Dolls]",[A long burnt orange coat is a staple to anyon...,Fits American Girl Dolls 18&quot; Town Coat Ou...,Emily Rose Doll Clothes,[Made to fit 18 Inch dolls such as American Gi...,B0058V4I68,Toys_and_Game


Video_Game (82394, 6)


,category,description,title,brand,feature,asin,domain
56890,"[Video Games, PC, </span></span></span>, New B...",[The Story\nDark Souls is the new action role-...,Dark Souls: Prepare to Die Edition,by\n \n Bandai,"[New Bosses Including Black Dragon, Sanctuary...",B00AJCRYZI,Video_Game
60880,"[Video Games, PlayStation 3, Accessories, Head...",[ASTRO Gaming is ready to invade the living ro...,ASTRO Gaming - A50 Wireless Headset [2013 model],by\n \n ASTRO Gaming,[5.8GHZ WIRELESS TECHNOLOGY Operating at a hig...,B00EIM3BV6,Video_Game


In [8]:
df = pd.concat(dfs)
df.shape

(13706665, 7)

In [9]:
df.sample(3)

,category,description,title,brand,feature,asin,domain
252223,"[Books, Romance, Contemporary]","[PAPERBACK, , ]",To Protect Their Child (Harlequin Intrigue 608),Sheryl Lynn,[],037322608X,Book
563450,"[Sports & Outdoors, Sports & Fitness, Exercise...",[<b>Keep Calm and Do YOGA</b> \n<br> \n<br> \n...,Yes4All HQ Yoga Block - 9 x 6 x 4 inch - Black...,Yes4All,"[High Density, Extremely Durable, and Studio Q...",B00IU5VSN0,Sports_and_Outdoor
1676838,"[Books, Health, Fitness &amp; Dieting, Diets &...","[, ]",Lose Belly Fat Diet: The Easy 14 Day Plan: Inc...,Sarah Davis,[],1500327530,Book


## cleanup

In [10]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [11]:
%%time
for col in ['description', 'feature']:
    df[col] = df[col].apply(lambda x: "\n".join(x) if len(x) else "")

CPU times: user 21.2 s, sys: 3.83 s, total: 25 s
Wall time: 25 s


In [12]:
df.fillna('', inplace=True)

In [13]:
for col in ['description', 'title', 'brand', 'feature',]:
    df[col] = df[col].apply(cleanhtml)

In [14]:
dct = df.sample().to_dict("records")[0]
for k,v in dct.items():
    print(f'-- {k} --')
    print(v)
    

-- category --
['Clothing, Shoes & Jewelry', 'Women', 'Shoes', 'Fashion Sneakers']
-- description --

-- title --
Vans Unisex's VANS ZAPATO DEL BARCO (C&P) SKATE SHOES 7.5 (NAVY)
-- brand --
Vans
-- feature --
Package Dimensions:
                    
12.4 x 5.3 x 5 inches
Shipping Weight:
                    
2.2 pounds
-- asin --
B0059BPVVI
-- domain --
Clothing_Shoes_and_Jewelry


## save

In [15]:
df.to_csv(f'../data/data__{prfx}.csv', index=False)

In [16]:
ls -hl ../data/data__{prfx}.csv

-rw-rw-r-- 1 ubuntu ubuntu 12G Mar  3 02:49 ../data/data__prep_20210302B1.csv


## make sample

In [17]:
import numpy as np
np.random.seed(101)
df_sample = df.sample(10000)

In [18]:
df_sample.to_csv(f'../data/data_sample__{prfx}.csv', index=False)